In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
path = '/content/drive/Shareddrives/통분방+공모전/데이터/'
csv_market = pd.read_csv(path+"전통시장현황_최종.csv", encoding="euc-kr")
csv_market.head()

,시장(상점가명),시장유형,주소(도로명),주소(지번),시장개설주기,점포수,취급품목,홈페이지주소,공중화장실보유여부,주차장보유여부
0,강남시장,상설장,서울특별시 강남구 압구정로 2길 46,서울특별시 강남구 신사동 510-11,매일,70,"농산물,축산물,가공식품,의류/신발,기타소매업",NaN,Y,Y
1,신사상가,상설장,서울특별시 강남구 압구정로29길 72-1,서울특별시 강남구 압구정동 454,매일,90,"농산물,수산물,가공식품,의류/신발",NaN,Y,N
2,영동전통시장,상설장,서울특별시 강남구 강남대로 128길 20 영동프라자 2동 204호,서울특별시 강남구 논현동 140,매일,110,"축산물,가공식품,의류/신발,기타소매업",NaN,Y,Y
3,청담삼익시장,상설장,서울특별시 강남구 학동로 101길 26,서울특별시 강남구 청담동 134-20,매일,183,"농산물,축산물,가공식품,의류/신발,기타소매업",NaN,Y,Y
4,고분다리전통시장,상설장,서울특별시 강동구 구천면로34길 13,서울특별시 강동구 천호동 393-32,매일,106,"농산물,축산물,수산물,가공식품,가정용품",NaN,Y,Y


In [ ]:
item = csv_market['취급품목']
item

0       농산물,축산물,가공식품,의류/신발,기타소매업
1             농산물,수산물,가공식품,의류/신발
2           축산물,가공식품,의류/신발,기타소매업
3       농산물,축산물,가공식품,의류/신발,기타소매업
4          농산물,축산물,수산물,가공식품,가정용품
                  ...           
1396                가공식품,근린생활서비스
1397                음식점업,근린생활서비스
1398                    농산물,가공식품
1399               음식점업,근린생활서비스업
1400         의류/신발,가정용품,근린생활서비스업
Name: 취급품목, Length: 1401, dtype: object

In [ ]:
agricultural = list()
livestock = list()
marine = list()

for i in range(len(item)):
    item_split = item[i].split(',')
    if '농산물' in item_split:
        agricultural.append("Y")
    else:
        agricultural.append("N")

for i in range(len(item)):
    item_split = item[i].split(',')
    if '축산물' in item_split:
        livestock.append("Y")
    else:
        livestock.append("N")
        
for i in range(len(item)):
    item_split = item[i].split(',')
    if '수산물' in item_split:
        marine.append("Y")
    else:
        marine.append("N")

In [ ]:
market_data = pd.DataFrame(zip(csv_market['시장(상점가명)'], csv_market['공중화장실보유여부'], csv_market['주차장보유여부'],
                              agricultural, livestock, marine, csv_market['점포수']),
                          columns=['시장명','화장실','주차장','농산물','축산물','수산물','점포수'])
market_data.head()

,시장명,화장실,주차장,농산물,축산물,수산물,점포수
0,강남시장,Y,Y,Y,Y,N,70
1,신사상가,Y,N,Y,N,Y,90
2,영동전통시장,Y,Y,N,Y,N,110
3,청담삼익시장,Y,Y,Y,Y,N,183
4,고분다리전통시장,Y,Y,Y,Y,Y,106


In [ ]:
# R에서 써볼라고 csv내보냄..
# https://cran.r-project.org/web/packages/FactoMineR/FactoMineR.pdf (FactoMineR 패키지 설명)
market_data.to_csv('C:\\Users\\samsung\\Desktop\\data\\market_data.csv')

# FAMD
https://en.wikipedia.org/wiki/Factor_analysis_of_mixed_data

Factor analysis of mixed data (FAMD) is a principal component method that combines principal component analysis (PCA) for continuous variables and multiple correspondence analysis (MCA) for categorical variables

In [ ]:
!pip install prince

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 참고 : https://github.com/MaxHalford/prince
# https://datascience.stackexchange.com/questions/9848/how-to-combine-pca-and-mca-on-mixed-data
import prince
X = market_data.drop('시장명', axis=1)
famd = prince.FAMD(n_components=2, n_iter=3, random_state=42).fit(X)
market = famd.transform(X)

In [ ]:
famd.explained_inertia_  # explained variance

array([0.34316023, 0.2338062 ])

In [ ]:
market # transformed data

,0,1
0,-0.292491,-0.047686
1,-0.342335,0.030893
2,0.351817,-0.664024
3,0.035541,0.154819
4,-0.574219,0.710389
...,...,...
1396,0.439100,-1.545473
1397,4.035845,0.674916
1398,0.317321,-0.606561
1399,0.292102,-1.298909


In [ ]:
famd.U_ # components

array([[-0.00744444, -0.0014704 ],
       [-0.00871306,  0.00095258],
       [ 0.00895438, -0.02047495],
       ...,
       [ 0.00807639, -0.0187031 ],
       [ 0.00743453, -0.04005143],
       [ 0.01775166, -0.04273614]])

In [ ]:
famd.U_.shape

(1401, 2)

In [ ]:
famd.column_correlations(X) # correlation between the original variables and the components

,0,1
농산물_N,0.199358,0.199214
농산물_Y,-0.199358,-0.199214
수산물_N,0.104855,0.104713
수산물_Y,-0.104855,-0.104713
점포수,1.000000,1.000000
주차장_N,-0.036306,-0.036291
주차장_Y,0.036306,0.036291
축산물_N,0.091243,0.091100
축산물_Y,-0.091243,-0.091100
화장실_N,-0.036043,-0.036025


In [ ]:
# https://stackoverflow.com/questions/70062663/finding-original-variable-names-of-the-important-attibutes-in-a-famd-pca-using-p
famd.V_ # contribute most to each component : 범주형 변수별로는 Y, N씩 loading / 수치형 변수는 단일 loading

array([[-0.04562909,  0.03080573,  0.27510229, -0.2148739 ,  0.19724611,
        -0.18997806,  0.20583074, -0.18898181, -0.05010067,  0.02362431,
         0.84792136],
       [ 0.0252007 , -0.01701384, -0.35834788,  0.27989446, -0.3706363 ,
         0.35697924, -0.36961128,  0.33935557,  0.03774687, -0.01779904,
         0.52344968]])

In [ ]:
# 0.27510229+0.2148739를 Y 대비 N 이런 식으로 해석하면,, 또 괜찮을 거 같기도 하고 ㅠㅠㅠㅠㅠㅠㅠㅠ 모르겠음